In [23]:
#untuk pake ini program ambil file ini dari laptop cuaca_data.csv, cuaca_data_regresi.csv, klasifikasi_cuaca.h5, prediksi_cuacaa.h5
#PREDIKSI DARI API
import requests
import json
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv('cuaca_data_regresi.csv', parse_dates=['time'])
data.set_index('time', inplace=True)
# Function to fetch weather data from API
def get_weather_data():
    api_url = "https://api.open-meteo.com/v1/forecast?latitude=-3.875&longitude=119.625&current=temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,rain,weather_code,cloud_cover,wind_speed_10m,wind_direction_10m"

    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for bad status codes
        weather_data = response.json()
        return weather_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None

# Load the saved model
model = load_model('prediksi_cuacaa.h5')

# Preprocess data (scale it)
def preprocess_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

# Define data columns (these should match the features used in the model)
data_columns = ["temperature_2m", "relative_humidity_2m", "apparent_temperature", "rain", "cloud_cover", "wind_speed_10m", "wind_direction_10m"]

# Get the weather data from the API
weather_info = get_weather_data()

if weather_info:
    try:
        # Extract relevant data from API response
        current_data = weather_info["current"]

        # Create user input dictionary
        user_input_weather = {
            "temperature_2m": current_data["temperature_2m"],
            "relative_humidity_2m": current_data["relative_humidity_2m"],
            "apparent_temperature": current_data["apparent_temperature"],
            "rain": current_data["rain"],
            "cloud_cover": current_data["cloud_cover"],
            "wind_speed_10m": current_data["wind_speed_10m"],
            "wind_direction_10m": current_data["wind_direction_10m"]
        }

        # Preprocess the weather data
        data = pd.DataFrame([user_input_weather])
        scaled_data, scaler = preprocess_data(data)

        # Prepare the input sequence for prediction (no need for seq_length > 1)
        seq_length = 1  # Predict just one step ahead
        padded_sequence = np.zeros((seq_length, len(data_columns)))  # Initialize empty sequence
        padded_sequence[-1, :] = scaled_data  # Add the current data as the last time step
        padded_sequence = np.expand_dims(padded_sequence, axis=0)  # Reshape for LSTM input

        # Predict weather one step ahead
        prediction_scaled = model.predict(padded_sequence)

        # Reverse scaling for the prediction
        predicted_weather = scaler.inverse_transform(prediction_scaled)[0]

        # Display the predicted weather for the next step
        predicted_result2 = dict(zip(data_columns, predicted_weather))
        print(f"Predicted weather for next step:")
        print(predicted_result2)

    except KeyError as e:
        print(f"Error accessing weather data: Missing key {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
Predicted weather for next step:
{'temperature_2m': 26.676193, 'relative_humidity_2m': 92.438736, 'apparent_temperature': 32.382607, 'rain': 0.0, 'cloud_cover': 100.03676, 'wind_speed_10m': 6.5774417, 'wind_direction_10m': 263.1799}


In [24]:
data2 = pd.read_csv('cuaca_data.csv', parse_dates=['time'])
data2.set_index('time', inplace=True)

In [25]:
data2 = data2.drop(columns=["precipitation (mm)"])

In [26]:
data2 = data2.drop(columns=["weather_code (wmo code)"])

In [27]:
data2.head()

,temperature_2m (°C),relative_humidity_2m (%),apparent_temperature (°C),rain (mm),cloud_cover (%),wind_speed_10m (km/h),wind_direction_10m (°)
time,,,,,,,
2004-01-01 00:00:00,24.4,91,29.1,0.0,99,4.4,305
2004-01-01 06:00:00,24.7,85,28.7,0.0,99,7.2,307
2004-01-01 12:00:00,29.1,67,31.7,0.0,86,17.3,272
2004-01-01 18:00:00,26.6,82,30.3,0.4,91,13.9,260
2004-01-02 00:00:00,24.8,92,29.6,0.0,98,7.0,282


In [28]:
data_column2 = ["temperature_2m", "relative_humidity_2m", "apparent_temperature", "rain", "cloud_cover", "wind_speed_10m", "wind_direction_10m"]

In [36]:
#HASIL PREDIKSI DIKLASIFIKASIKAN
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd


# Load model
scaler2 = StandardScaler()
model = load_model("klasifikasi_cuaca.h5")

# Input baru (pastikan sesuai dengan fitur model)

new_data_raw = np.array([[25, 93, 28, 0.0, 81, 5, 80]])  # Data mentah
# new_data_raw = np.array([[predicted_result2[col] for col in data_column2]])

# Assuming you have historical data in a pandas DataFrame called 'data'
# Replace 'data' with the actual name of your DataFrame
# Fit the scaler using your historical data
scaler2.fit(data2) #This line is added to address the NotFittedError

# Lakukan scaling pada input
new_data_scaled = scaler2.transform(new_data_raw)

# Prediksi
predictions = model.predict(new_data_scaled)

# Konversi dari one-hot encoding ke kelas asli
predicted_class = np.argmax(predictions, axis=1)

print(f"Prediksi Kelas: {predicted_class}")
print(f"Probabilitas untuk Setiap Kelas: {predictions}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Prediksi Kelas: [3]
Probabilitas untuk Setiap Kelas: [[0.00000000e+00 3.20582247e-20 6.82970509e-02 9.29319680e-01
  4.21495646e-19 1.08060241e-16 4.82875418e-17 1.20448907e-14
  5.43352445e-18 6.83031363e-18 2.85477169e-17 1.27670788e-18
  1.12423978e-17 6.25807670e-18 1.21806657e-18 1.12550678e-15
  1.05080135e-16 2.90456278e-19 1.27370277e-16 3.39928245e-16
  3.30570066e-16 5.97435261e-16 7.87710366e-18 7.23638577e-17
  1.96899229e-14 1.73543146e-13 3.01608104e-16 4.94335666e-18
  3.76410683e-19 1.76055269e-17 5.07417445e-18 3.49375553e-15
  7.52715193e-16 3.71495350e-17 3.76078096e-17 1.31012455e-17
  4.21800305e-16 3.07971227e-16 2.88234691e-17 2.98564125e-17
  1.63342727e-17 4.13143397e-17 1.71160887e-16 6.75040724e-17
  1.78200631e-16 1.54440493e-17 1.67023198e-16 1.24334352e-16
  2.09865863e-17 1.28782674e-17 2.24602258e-19 2.38326495e-03
  1.81083547e-16 5.01167057e-34 6.26519500e-18 6.34089805e-29
  3.96996468e-17 3.10050756e-15 1.4488881

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
